## Imports


In [1]:
import pandas as pd
import joblib
import altair as alt

/Users/jasminkazi/.pyenv/versions/3.8.6/envs/fydjob/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
data = joblib.load('/Users/jasminkazi/Downloads/ip_2021-03-03.joblib')

### Exploration


In [3]:
data.shape

(954, 11)

In [4]:
data.head()

,job_title,job_text,company,location,job_info,query_text,tag_language,job_info_tokenized,job_text_tokenized,job_text_tokenized_titlecase,job_title_tokenized
0,Data Scientist / Matching Engineer (m/w/d),You are responsible for improvement of Taledo’...,Taledo,Berlin,Data Scientist / Matching Engineer (m/w/d)\nTa...,data engineer,en,"[data, scientist, matching, engineer, mwd, tal...","[you, are, responsible, for, improvement, of, ...","[You, are, responsible, for, improvement, of, ...","[data, scientist, matching, engineer, mwd]"
1,(Junior) Data Engineer (f/m/x),Über die Stelle\nUnser Data Team braucht Unter...,Customlytics GmbH,Berlin,(Junior) Data Engineer (f/m/x)\nCustomlytics G...,data engineer,de,"[junior, data, engineer, fmx, customlytics, gm...","[über, die, stelle, unser, data, team, braucht...","[Über, die, Stelle, Unser, Data, Team, braucht...","[junior, data, engineer, fmx]"
2,Junior Data Scientist (m/w/d) docmetric,Kennziffer:\nreq6004\nStandort:\nBerlin\nJob S...,CompuGroup Medical,Berlin,Junior Data Scientist (m/w/d) docmetric\nCompu...,data engineer,de,"[junior, data, scientist, mwd, docmetric, comp...","[kennziffer, req, standort, berlin, job, segme...","[Kennziffer, req, Standort, Berlin, Job, Segme...","[junior, data, scientist, mwd, docmetric]"
3,Senior Software Engineer - Data Platform,We are looking for a Senior Software Engineer ...,Zalando SE,Berlin,Senior Software Engineer - Data Platform\nZala...,data engineer,en,"[senior, software, engineer, data, platform, z...","[we, are, looking, for, a, senior, software, e...","[We, are, looking, for, a, Senior, Software, E...","[senior, software, engineer, data, platform]"
4,Werkstudent Software/Data Engineer (w/m/d) | J...,bakdata GmbH\nDie bakdata GmbH ist ein unabhän...,bakdata GmbH,Berlin,Werkstudent Software/Data Engineer (w/m/d) | J...,data engineer,de,"[werkstudent, softwaredata, engineer, wmd, jav...","[bakdata, gmbh, die, bakdata, gmbh, ist, ein, ...","[bakdata, GmbH, Die, bakdata, GmbH, ist, ein, ...","[werkstudent, softwaredata, engineer, wmd, jav..."


In [5]:
data.columns

Index(['job_title', 'job_text', 'company', 'location', 'job_info',
       'query_text', 'tag_language', 'job_info_tokenized',
       'job_text_tokenized', 'job_text_tokenized_titlecase',
       'job_title_tokenized'],
      dtype='object')

### No. of...


In [6]:
#Lanugauges
data['tag_language'].nunique()
source_lang = pd.DataFrame(data['tag_language'].value_counts()).reset_index()
source_lang = source_lang.rename(columns = {'index':'Posting Language', 'tag_language':'Count'})
#pd.DataFrame(data['tag_language'].value_counts(), columns=['language', 'count'])
source_lang

,Posting Language,Count
0,en,707
1,de,247


In [7]:
alt.Chart(source_lang).mark_bar().encode(
    x=alt.X('Count', stack ='normalize'),
    #y='Posting Language',
    color = 'Count'
)


alt.Chart(...)

In [8]:
alt.Chart(source_lang).transform_joinaggregate(
    Total='sum(Count)',
).transform_calculate(
    Percentage="datum.Count / datum.Total"
).mark_bar().encode(
    alt.X('Percentage:Q', axis=alt.Axis(format='.0%')),
    y='Posting Language:N'
)

alt.Chart(...)

In [9]:
#Companies
print('Hiring Companies: ', data['company'].nunique())
source_comp = pd.DataFrame(data['company'].value_counts().reset_index())
source_comp= source_comp.rename(columns = {'index':'Company', 'company':'Count'})
source_comp

Hiring Companies:  454


,Company,Count
0,Zalando,42
1,Zalando SE,35
2,Delivery Hero,34
3,Amazon Development Center DEU,27
4,,11
...,...,...
449,On AG,1
450,GHD,1
451,Axel Springer hy GmbH,1
452,Digital Rocks GmbH,1


In [10]:
source_comp = source_comp.nlargest(30, 'Count')

bars = alt.Chart(source_comp).mark_bar().encode(
    x='Count:Q',
    y=alt.Y('Company:N', sort='-x')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Count:Q'
)

(bars + text).properties(height=500)

alt.LayerChart(...)

In [11]:
#query_text
print('No of query_text: ',data['query_text'].nunique())
source_job = pd.DataFrame(data['query_text'].value_counts().reset_index())
source_job= source_job.rename(columns = {'index':'Job', 'query_text':'Count'})
source_job

No of query_text:  4


,Job,Count
0,data science,281
1,data engineer,274
2,data scientist,229
3,business intelligence,170


In [12]:
alt.Chart(source_job).mark_bar().encode(
    x='Count',
    y='Job'
)
#rule = alt.Chart(source_job).mark_rule(color='red').encode(
#    x='mean(Count):Q'
#)

alt.Chart(...)

In [13]:
type(data['job_text_tokenized'][0])

list

In [14]:
from fydjob import utils
utils.load_skills()     #loads the skills from JSON file


{'business': ['accounting',
  'marketing',
  'sales',
  'distribution',
  'logistic',
  'scm',
  'hr',
  'administration',
  'administrative',
  'management',
  'planning',
  'strategy',
  'strategic',
  'strategical',
  'bd',
  'reporting',
  'report',
  'visualization',
  'visualisation',
  'kpi',
  'business',
  'stakeholder',
  'client',
  'industry',
  'entrepreneurship',
  'entrepreneur',
  'entrepreneurial',
  'consulting',
  'consult',
  'analyst',
  'analyze',
  'analyse',
  'analysis',
  'analytical',
  'analytics',
  'analytic',
  'solution',
  'olap',
  'feasibility',
  'measurable',
  'profitable',
  'commercial',
  'crm',
  'efficiency',
  'advertising',
  'managing',
  'dashboards'],
 'knowledge': ['mathematics',
  'science',
  'bayes',
  'bayesian',
  'bioinformatics',
  'biostatistics',
  'blockchain',
  'boosting',
  'causality',
  'chatbot',
  'chi',
  'classification',
  'cleaning',
  'cleansing',
  'cluster',
  'clustering',
  'cnn',
  'convolutional',
  'correlati

In [15]:
# get all the skill sets
categories = utils.load_skills()
categories.keys()
bus_skills = set(utils.load_skills()['business'])
knowledge_skills = set(utils.load_skills()['knowledge'])
code_skills = set(utils.load_skills()['programming'])
soft_skills = set(utils.load_skills()['soft_skills'])


In [16]:
#get skill for the first token
skill_0 = []

for word in data['job_text_tokenized'][0]:
    if word in code_skills:
        skill_0.append(word)
        
skill_0

['python',
 'numpy',
 'pandas',
 'dask',
 'scikitlearn',
 'xgboost',
 'keras',
 'pytorch',
 'spacy',
 'seaborn',
 'matplotlib',
 'elasticsearch']

In [17]:
# count the occurences
from collections import Counter
counter0 = Counter(data['job_text_tokenized'][0])

#occs = {x: counter[x] for x in counter.keys() & set(skill_0)}

bus_occs = {x: counter0[x] for x in counter0.keys() & set(bus_skills)}
code_occs = {x: counter0[x] for x in counter0.keys() & set(code_skills)}
knowledge_occs ={x: counter0[x] for x in counter0.keys() & set(knowledge_skills)}
soft_occs ={x: counter0[x] for x in counter0.keys() & set(soft_skills)}
    
print(bus_occs)
print(code_occs)
print(knowledge_occs)
print(soft_occs)


{'marketing': 1, 'solution': 1, 'industry': 1, 'management': 1, 'business': 1, 'visualization': 1}
{'dask': 1, 'xgboost': 1, 'matplotlib': 1, 'scikitlearn': 1, 'seaborn': 1, 'spacy': 1, 'keras': 1, 'python': 1, 'elasticsearch': 1, 'pandas': 1, 'numpy': 1, 'pytorch': 1}
{'research': 1, 'nlp': 1, 'ai': 1, 'visualization': 1, 'science': 1}
{'proactive': 1, 'trustful': 1, 'management': 1, 'flexible': 1, 'responsible': 1, 'responsibility': 1, 'creative': 1}


In [18]:
jd_0 = set(data['job_text_tokenized'][5])
vacancy = data['job_text_tokenized'][5]
jd_0

{'a',
 'ability',
 'academy',
 'achieve',
 'adding',
 'additional',
 'ahead',
 'all',
 'ambitious',
 'an',
 'and',
 'api',
 'applications',
 'architecture',
 'are',
 'as',
 'assistance',
 'at',
 'automation',
 'available',
 'aws',
 'backgrounds',
 'basis',
 'beverages',
 'bewerbung',
 'bigquery',
 'blogs',
 'build',
 'building',
 'business',
 'by',
 'celebrate',
 'centrally',
 'child',
 'class',
 'cloud',
 'code',
 'come',
 'commitment',
 'committed',
 'competitive',
 'compliance',
 'constructive',
 'consult',
 'cost',
 'create',
 'creating',
 'culture',
 'data',
 'dataprocessing',
 'decide',
 'decided',
 'deliver',
 'demos',
 'design',
 'develop',
 'development',
 'diesen',
 'discount',
 'discounts',
 'distributed',
 'diverse',
 'diversity',
 'efficiency',
 'employee',
 'employment',
 'empowerment',
 'engineer',
 'engineering',
 'equipment',
 'equivalent',
 'events',
 'exasol',
 'excellence',
 'experience',
 'experienced',
 'expert',
 'expertise',
 'experts',
 'extensive',
 'external'

In [19]:
jd_0.intersection(bus_skills)

{'business', 'consult', 'efficiency', 'industry', 'strategy'}

In [20]:
jd_0.intersection(code_skills)

{'aws', 'bigquery', 'cloud', 'hive', 'lambda', 'python', 'spark'}

In [21]:
jd_0.intersection(soft_skills)

{'committed', 'constructive', 'flexible', 'time', 'volunteering'}

In [22]:
jd_0.intersection(knowledge_skills)

{'api', 'optimizing'}

In [23]:
for skill in categories:
    print(skill, jd_0.intersection(categories[skill]))


business {'industry', 'efficiency', 'business', 'consult', 'strategy'}
knowledge {'api', 'optimizing'}
programming {'lambda', 'cloud', 'bigquery', 'spark', 'hive', 'python', 'aws'}
soft_skills {'committed', 'time', 'volunteering', 'flexible', 'constructive'}
tech_adjectives {'architecture', 'google', 'integration'}


In [24]:
intersects_know = jd_0.intersection(knowledge_skills)
intersects_code = jd_0.intersection(code_skills)
intersects_soft = jd_0.intersection(soft_skills)
intersect_bus = jd_0.intersection(bus_skills)

In [25]:
vacancy.count('xgboost')

0

In [26]:
occsurrences_know ={} 
for skill in intersects_know:
    occsurrences_know[skill] = vacancy.count(skill)
occsurrences_know


{'api': 1, 'optimizing': 1}

In [27]:
occsurrences_code ={} 
for skill in intersects_code:
    occsurrences_code[skill] = vacancy.count(skill)
occsurrences_code

{'lambda': 1,
 'cloud': 4,
 'bigquery': 1,
 'spark': 2,
 'hive': 1,
 'python': 1,
 'aws': 4}

In [28]:
occsurrences_soft ={} 
for skill in intersects_soft:
    occsurrences_soft[skill] = vacancy.count(skill)
occsurrences_soft

{'committed': 1,
 'time': 1,
 'volunteering': 1,
 'flexible': 1,
 'constructive': 1}

In [29]:
occsurrences_bus ={} 
for skill in intersects_soft:
    occsurrences_bus[skill] = vacancy.count(skill)
occsurrences_bus


{'committed': 1,
 'time': 1,
 'volunteering': 1,
 'flexible': 1,
 'constructive': 1}

In [30]:
categories

{'business': ['accounting',
  'marketing',
  'sales',
  'distribution',
  'logistic',
  'scm',
  'hr',
  'administration',
  'administrative',
  'management',
  'planning',
  'strategy',
  'strategic',
  'strategical',
  'bd',
  'reporting',
  'report',
  'visualization',
  'visualisation',
  'kpi',
  'business',
  'stakeholder',
  'client',
  'industry',
  'entrepreneurship',
  'entrepreneur',
  'entrepreneurial',
  'consulting',
  'consult',
  'analyst',
  'analyze',
  'analyse',
  'analysis',
  'analytical',
  'analytics',
  'analytic',
  'solution',
  'olap',
  'feasibility',
  'measurable',
  'profitable',
  'commercial',
  'crm',
  'efficiency',
  'advertising',
  'managing',
  'dashboards'],
 'knowledge': ['mathematics',
  'science',
  'bayes',
  'bayesian',
  'bioinformatics',
  'biostatistics',
  'blockchain',
  'boosting',
  'causality',
  'chatbot',
  'chi',
  'classification',
  'cleaning',
  'cleansing',
  'cluster',
  'clustering',
  'cnn',
  'convolutional',
  'correlati

In [31]:
for category in categories:
    print(category)

business
knowledge
programming
soft_skills
tech_adjectives


In [32]:
vacancy = data['job_text_tokenized'][5]
vacancy_set = set(vacancy)
matching_skill_per_category = {}

for category in categories:
    matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
#print(matching_skill_per_category)
    
occ = {}
for category in categories:
    occ[category] = {}
    for s in matching_skill_per_category[str(category)]:
        occ[category][s] =  vacancy.count(s)
print(occ)


{'business': {'industry': 1, 'efficiency': 1, 'business': 2, 'consult': 1, 'strategy': 1}, 'knowledge': {'api': 1, 'optimizing': 1}, 'programming': {'lambda': 1, 'cloud': 4, 'bigquery': 1, 'spark': 2, 'hive': 1, 'python': 1, 'aws': 4}, 'soft_skills': {'committed': 1, 'time': 1, 'volunteering': 1, 'flexible': 1, 'constructive': 1}, 'tech_adjectives': {'architecture': 1, 'google': 2, 'integration': 2}}


In [33]:
data['job_text_tokenized']

0       [you, are, responsible, for, improvement, of, ...
1       [über, die, stelle, unser, data, team, braucht...
2       [kennziffer, req, standort, berlin, job, segme...
3       [we, are, looking, for, a, senior, software, e...
4       [bakdata, gmbh, die, bakdata, gmbh, ist, ein, ...
                              ...                        
1418    [die, gries, deco, company, gmbh, hat, sich, i...
1426    [per, sofort, oder, nach, vereinbarung, suchen...
1445    [about, the, position, youll, be, the, first, ...
1451    [firmenbeschreibung, in, rosenheim, gegründet,...
1462    [we, digitize, the, logistics, industry, a, bn...
Name: job_text_tokenized, Length: 954, dtype: object

In [34]:
occ

{'business': {'industry': 1,
  'efficiency': 1,
  'business': 2,
  'consult': 1,
  'strategy': 1},
 'knowledge': {'api': 1, 'optimizing': 1},
 'programming': {'lambda': 1,
  'cloud': 4,
  'bigquery': 1,
  'spark': 2,
  'hive': 1,
  'python': 1,
  'aws': 4},
 'soft_skills': {'committed': 1,
  'time': 1,
  'volunteering': 1,
  'flexible': 1,
  'constructive': 1},
 'tech_adjectives': {'architecture': 1, 'google': 2, 'integration': 2}}

In [35]:
for vacancy in data['job_text_tokenized'][:3]:
    set(vacancy)

In [36]:
type(data['job_text_tokenized'].values)

numpy.ndarray

In [37]:
categories

{'business': ['accounting',
  'marketing',
  'sales',
  'distribution',
  'logistic',
  'scm',
  'hr',
  'administration',
  'administrative',
  'management',
  'planning',
  'strategy',
  'strategic',
  'strategical',
  'bd',
  'reporting',
  'report',
  'visualization',
  'visualisation',
  'kpi',
  'business',
  'stakeholder',
  'client',
  'industry',
  'entrepreneurship',
  'entrepreneur',
  'entrepreneurial',
  'consulting',
  'consult',
  'analyst',
  'analyze',
  'analyse',
  'analysis',
  'analytical',
  'analytics',
  'analytic',
  'solution',
  'olap',
  'feasibility',
  'measurable',
  'profitable',
  'commercial',
  'crm',
  'efficiency',
  'advertising',
  'managing',
  'dashboards'],
 'knowledge': ['mathematics',
  'science',
  'bayes',
  'bayesian',
  'bioinformatics',
  'biostatistics',
  'blockchain',
  'boosting',
  'causality',
  'chatbot',
  'chi',
  'classification',
  'cleaning',
  'cleansing',
  'cluster',
  'clustering',
  'cnn',
  'convolutional',
  'correlati

In [38]:
#vacancy = data['job_text_tokenized'][5]


def get_skill_count(vacancy):
    # turn the vacancy into a set to compare the words with defined dictionary
    vacancy_set = set(vacancy)
    # get the matching skills of a job description to the skills defined within different categories of skillsets (tech, bus, softskill...)
    matching_skill_per_category = {}
    for category in categories:
        matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
    
    #count the occurences of every matching skill
    occurrences = {}
    for category in categories:
        occurrences[category] = {}
        for skill in matching_skill_per_category[str(category)]:
            occurrences[category][skill] =  vacancy.count(skill)
    return occurrences

In [39]:
requirement = {}
for index, vacancy in enumerate(data['job_text_tokenized']):
    #print(vacancy)
    requirement[index]= get_skill_count(vacancy)
    occurence =get_skill_count(vacancy)
    

In [40]:
requirement

{0: {'business': {'marketing': 1,
   'solution': 1,
   'industry': 1,
   'management': 1,
   'business': 1,
   'visualization': 1},
  'knowledge': {'research': 1,
   'nlp': 1,
   'ai': 1,
   'visualization': 1,
   'science': 1},
  'programming': {'dask': 1,
   'xgboost': 1,
   'matplotlib': 1,
   'scikitlearn': 1,
   'seaborn': 1,
   'keras': 1,
   'spacy': 1,
   'python': 1,
   'elasticsearch': 1,
   'pandas': 1,
   'numpy': 1,
   'pytorch': 1},
  'soft_skills': {'proactive': 1,
   'trustful': 1,
   'management': 1,
   'flexible': 1,
   'responsible': 1,
   'responsibility': 1,
   'creative': 1},
  'tech_adjectives': {'algorithmic': 2, 'algorithmical': 2, 'metric': 1}},
 1: {'business': {'marketing': 10,
   'analyst': 1,
   'crm': 1,
   'stakeholder': 1,
   'industry': 1,
   'analytics': 3,
   'dashboards': 1,
   'consulting': 1,
   'reporting': 1,
   'analyse': 2},
  'knowledge': {'hardware': 1, 'optimization': 1},
  'programming': {'cloud': 1,
   'bigquery': 1,
   'sql': 1,
   'git'

In [41]:
#things to do for streamlit

# ranking of all skills for streamlit --> agregated view of each categ
# percentage of category per job offer (10% bus, 20% tech...)

In [42]:
#put all vacancies into one huge list
all_vacancies=[]
for job in data['job_text_tokenized']:
    #print(len(job))
    all_vacancies = all_vacancies + job

In [43]:
len(all_vacancies)

468225

In [44]:
vacancy_set = set(all_vacancies)
len(vacancy_set)

21298

In [45]:

matching_skill_per_category = {}

for category in categories:
    matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
#print(matching_skill_per_category)
occ = {}
for category in categories:
    occ[category] = {}
    for s in matching_skill_per_category[str(category)]:
        occ[category][s] =  all_vacancies.count(s)
print(occ)

{'business': {'advertising': 132, 'industry': 338, 'managing': 119, 'analytics': 663, 'consult': 39, 'visualization': 80, 'reporting': 210, 'strategical': 2, 'entrepreneur': 7, 'commercial': 80, 'marketing': 605, 'accounting': 43, 'stakeholder': 55, 'kpi': 15, 'business': 1892, 'measurable': 8, 'sales': 203, 'client': 117, 'olap': 2, 'administration': 50, 'distribution': 24, 'analytical': 292, 'analyse': 127, 'feasibility': 11, 'administrative': 10, 'analyst': 121, 'crm': 82, 'entrepreneurial': 67, 'visualisation': 20, 'profitable': 16, 'efficiency': 58, 'dashboards': 88, 'hr': 51, 'report': 46, 'analyze': 101, 'solution': 122, 'management': 724, 'analysis': 372, 'logistic': 9, 'analytic': 24, 'consulting': 125, 'planning': 113, 'entrepreneurship': 5, 'strategic': 148, 'strategy': 225}, 'knowledge': {'cleaning': 12, 'nlp': 55, 'lstm': 7, 'visualization': 80, 'opensource': 22, 'saas': 57, 'cicd': 52, 'api': 24, 'hypothesis': 9, 'reinforcement': 3, 'segmentation': 25, 'pipeline': 74, 'nv

In [46]:
def get_skill_aggr(all_vacancies):
    
    vacancy_set = set(all_vacancies)
    matching_skill_per_category = {}

    for category in categories:
        matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
    
    occ = {}
    for category in categories:
        occ[category] = {}
        for s in matching_skill_per_category[str(category)]:
            occ[category][s] =  all_vacancies.count(s)
    return occ

In [47]:
all_vacancies=[]
for job in data['job_text_tokenized']:
    #print(len(job))
    all_vacancies = all_vacancies + job
total_occurences = get_skill_aggr(all_vacancies)

In [48]:
df = pd.DataFrame(total_occurences).reset_index()
df= df.rename(columns = {'index':'skill'})

df

,skill,business,knowledge,programming,soft_skills,tech_adjectives
0,advertising,132.0,NaN,NaN,NaN,NaN
1,industry,338.0,NaN,NaN,NaN,NaN
2,managing,119.0,NaN,NaN,NaN,NaN
3,analytics,663.0,NaN,NaN,NaN,NaN
4,consult,39.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
359,maintainable,NaN,NaN,NaN,NaN,21.0
360,scientific,NaN,NaN,NaN,NaN,198.0
361,algorithmical,NaN,NaN,NaN,NaN,3.0
362,gerrit,NaN,NaN,NaN,NaN,1.0


In [74]:
if df.loc[df['business'] > 0]:
    df['category'] = 'business'

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [68]:
for category in df.columns[1:]:
    #print(df[category])
    if df.loc[df[category] > 0,:]:
              df['category']=category
#df.loc[df['a'] > 10, ['a','c']]

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df_bus = df[['skill','business']].dropna().sort_values('business', ascending = False)
df_bus

In [ ]:
movies = alt.UrlData(
    data.movies.url,
    format=alt.DataFormat(parse={"Release_Date":"date"})
)
ratings = ['G', 'NC-17', 'PG', 'PG-13', 'R']
genres = ['Action', 'Adventure', 'Black Comedy', 'Comedy',
       'Concert/Performance', 'Documentary', 'Drama', 'Horror', 'Musical',
       'Romantic Comedy', 'Thriller/Suspense', 'Western']

base = alt.Chart(movies, width=200, height=200).mark_point(filled=True).transform_calculate(
    Rounded_IMDB_Rating = "floor(datum.IMDB_Rating)",
    Hundred_Million_Production =  "datum.Production_Budget > 100000000.0 ? 100 : 10",
    Release_Year = "year(datum.Release_Date)"
).transform_filter(
    alt.datum.IMDB_Rating > 0
).transform_filter(
    alt.FieldOneOfPredicate(field='MPAA_Rating', oneOf=ratings)
).encode(
    x=alt.X('Worldwide_Gross:Q', scale=alt.Scale(domain=(100000,10**9), clamp=True)),
    y='IMDB_Rating:Q',
    tooltip="Title:N"
)

# A dropdown filter
genre_dropdown = alt.binding_select(options=Category)
genre_select = alt.selection_single(fields=['Major_Genre'], bind=genre_dropdown, name="Genre")

filter_genres = base.add_selection(
    genre_select
).transform_filter(
    genre_select
).properties(title="Dropdown Filtering")


In [ ]:
list(df.columns)

In [ ]:
source = df
category = list(df.columns)


base= alt.Chart(source).mark_bar().encode(
    x=alt.X('business'),
    y=alt.Y('skill:N', sort='-x'))
base